# 23장 오프라인 애플리케이션과 클라이언트 사이드 스토리지  

### 이 장에서 다루는 내용  
- 오프라인 탐지 설정
- 오프라인 캐시 사용
- 브라우저에 데이터 저장 

HTML5에서 오프라인 웹 애플리케이션을 도입했다.  
'오프라인' 웹 애플리케이션은 장치가 인터넷에 연결되어 있지 않아도 동작한다.  


## 23.1 오프라인 탐지  

오프라인 애플리케이션을 만드는 첫 단계가 장치의 오프라인 여부를 구분하는 것이므로 HTML5 에서는 navigator.onLine 프로퍼티를 정의하여 인터넷에 연결되어 있을 때 True, 그렇지 않을 때는 False를 반환하게 했다.(브라우저마다 조금씩 다르게 동작함)

```javascript
if (navigator.onLine) {
    // 평상시대로 동작
} else {
    // 오프라인 동작 
}
```


HTML5에서는 navigator.onLine 외에도 네트워크를 추적하는 online과 offline 두가지 이벤트를 정의했다.  

```javascript
window.addEventListner('online', function () {
    alert('online!');
})


window.addEventListner('offline', function () {
    alert('offline!');
})
```


## 23.2 애플리케이션 캐시  


[HTML5 어플리케이션 캐시 사용하기](https://devbobos.github.io/2018/03/20/HTML5-%EC%96%B4%ED%94%8C%EB%A6%AC%EC%BC%80%EC%9D%B4%EC%85%98-%EC%BA%90%EC%8B%9C-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0.html)
  
HTML5의 '애플리케이션 캐시'는 'appcache'라고 부르기도 하며 오프라인 웹 애플리케이션을 위해 디자인 되었다.  
애플리케이션 캐시는 일반적인 브라우저 캐시와 분리된 영역에 존재하는 캐시이다.  
페이지의 애플리케이션 캐시에 어떤 자원을 저장할지는 '매니페스트 파일'에 지정한다.  

가장 단순한 매니페스트 파일은 오프라인에서 사용할 자원 목록을 나열한 형태이다.  
```javascript
CACHE MANIFEST
#Comment

file.js
file.css
```


매니페스트 파일 경로는 다음과 같이 html manifest 속성에 지정한다.  
```html
<html manifest="/offline.manifest">
```
    

## 23.3 데이터 스토리지 

웹 애플리케이션이 발전하면서 사용자 정보를 클라이언트에 저장할 필요성도 늘었다.  


### 23.3.1 쿠키  

[HTTP 쿠키](https://developer.mozilla.org/ko/docs/Web/HTTP/Cookies)

HTTP 쿠키는 대부분 '쿠키'라고만 부르는데 원래 의도는 세션 정보를 클라이언트에 저장하는 것이였다.  
명세에서는 서버에서 Set-Cookie라는 HTTP 헤더에 세션 정보를 담아서 모든 HTTP 응답에 포함하도록 규정했다.  

```javascript
    // 서버 응답 헤더
    HTTP/1.1 200 OK
    Content-type: text/html
    Set-Cookie: name=value
    Other-header: other-header-value

    // 클라이언트 요청 헤더를 통해 서버는 어디서 요청이 왔는지를 구분한다. 
    GET /index.html HTTP/1.1
    Cookie: name=value
    Other-header: other-header-value
```


아래는 쿠키 생성, 삭제, 읽어오는 방법이다. 
```javascript

    // 쿠키 생성
    const date = new Date();
    date.setDate(date.getDate() + 7);

    let willCookie = "";
    willCookie += "CookieName=Value;";
    willCookie += "expires=" + date.toUTCString();

    // 쿠키에 넣는다.
    document.cookie = willCookie;

```


```javascript

    // 쿠키값을 읽어오는 함수 생성
    const getCookieValue = (key) => {
      let cookieKey = key + "="; 
      let result = "";
      const cookieArr = document.cookie.split(";");

      for(let i = 0; i < cookieArr.length; i++) {
        if(cookieArr[i][0] === " ") {
          cookieArr[i] = cookieArr[i].substring(1);
        }

        if(cookieArr[i].indexOf(cookieKey) === 0) {
          result = cookieArr[i].slice(cookieKey.length, cookieArr[i].length);
          return result;
        }
      }
      return result;
    }
    
   
    getCookieValue('CookieName');

```


```javascript

    // 쿠키 제거, Expires 속성을 이전의 날짜로 설정하면 된다.  
    const date = new Date();
    date.setDate(date.getDate() - 1);

    let willCookie = "";
    willCookie += "CookieName=Value;";
    willCookie += "expires=" + date.toUTCString();

    // 쿠키에 넣는다.
    document.cookie = willCookie;

```



#### 쿠키에서 고려할 점   
쿠키 중에는 'HTTP 전용' 쿠키도 있다.  
HTTP 전용 쿠키는 브라우저나 서버 양쪽에서 설정 가능하지만 자바스크립트에서는 그 값을 읽을 수 없어서 서버에서만 사용 가능한 쿠키이다.  
브라우저에서 보내는 요청 헤더에는 모두 쿠키가 포함되므로, 쿠키에 지나치게 많은 정보를 저장하면 해당 도메인에 대한 요청 성능에 문제가 생길 수 있다.  
쿠키의 본성과 각종 제한 때문에 많은 양의 정보를 저장하기에 이상적이라 할 수 없으므로 다른 방법이 등장하였다.  


### 23.3.3 웹 스토리지  

[웹 스토리지](http://www.codejs.co.kr/html5-web-storage/#:~:text=Web%20Storage%EB%9E%80%20%ED%81%B4%EB%9D%BC%EC%9D%B4%EC%96%B8%ED%8A%B8%EC%9D%98,%EC%9C%BC%EB%A1%9C%20Cookie%EB%A5%BC%20%EC%82%AC%EC%9A%A9%ED%95%98%EC%98%80%EC%8A%B5%EB%8B%88%EB%8B%A4.)  

[쿠키, 세션스토리지, 로컬스토리지](https://velog.io/@ejchaid/localstorage-sessionstorage-cookie%EC%9D%98-%EC%B0%A8%EC%9D%B4%EC%A0%90)

웹 스토리지의 우선적 목표는 두가지이다.  
- 세션 데이터를 쿠키 바깥에 저장할 방법을 제공한다.  
- 대량의 데이터를 세션이 끝나도 유지할 메커니즘을 제공한다.  


#### Storage 타입  

Storage 타입은 이름-값 쌍을 담도록 만들어졌으며 최댓값은 브라우저에 다라 다르다.  
Storage의 인스턴스는 다른 객체와 마찬가지로 동작하며 다음 메서드가 추가되어 있다. 
- clear(): 모든 값을 제거
- getItem(name): 주어진 name에 해당하는 값 가져오기
- key(index): 주어진 인덱스의 이름을 가져오기
- removeItem(name): name에 해당하는 이름-값 쌍을 제거
- setItem(name, vale): 주어진 name에 값을 설정

Storage는 크게 두가지로 나눈다. 
- sessionStorage: 세션이 유지되는 동안만 데이터를 저장하며 저장된 데이터는 브라우저를 다을 때 모두 사라진다.  
- localStorage: 저장한 데이터를 명시적으로 지우지 않는 이상 영구적으로 저장한다.


#### 제약과 한계  
다른 클라이언트 데이터 스토리지 솔루션과 마찬가지로 웹 스토리지에도 한계가 존재한다.  
이들 제한은 브라우저에 따라 다르다.  
데스크톱 브라우저는 대게 localStorage를 소스당 5MB로 제한한다.  
크롬과 사파리는 2.5MB이다.  


### 23.3.4 IndexedDB  

[IndexedDB](https://developer.mozilla.org/ko/docs/Web/API/IndexedDB_API/Basic_Concepts_Behind_IndexedDB)

[IndexedDB 사용하기](https://developer.mozilla.org/ko/docs/Web/API/IndexedDB_API/Using_IndexedDB)

인덱스된 데이터베이스 API는 'IndexedDB'라는 약자로 칭하며 브라우저에서 사용하는 구조화된 데이터 저장소이다.  
IndexedDB는 현재 폐기된 웹 SQL 데이터 베이스 API의 대안으로 개발되었다.  
IndexDB를 완전히 지원하는 브라우저는 전역 indexedDB 객체를 지원하며 이 객체가 API 호스트 역할을 한다.  
브라우저 제조사들은 객체에 접두사를 붙여 구현한다.  

```javascript
window.indexedDB = window.indexedDB || window.mozIndexedDB || window.webkitIndexedDB || window.msIndexedDB;
```


#### 데이터베이스  
IndexedDB는 MySql이나 웹 SQL 데이터베이스와 비슷한 데이터베이스이다.  
가장 큰 차이는 IndexedDB에서 테이블 대신 객체 저장소를 이용해 데이터를 저장한다는 것이다.  

```javascript
var db;
var request = indexedDB.open("MyTestDatabase");

request.onerror = function(event) {
    // request.errorCode 에 대해 무언가를 한다!
    alert("Why didn't you allow my web app to use IndexedDB?!");
};

request.onsuccess = function(event) {
    // request.result 에 대해 무언가를 한다!
    db = request.result;
};
```   

#### 객체 저장소  
데이터베이스 연결을 수립한 다음 단계는 객체 저장소와의 상호작용이다.  
사용자 아이디와 비밀번호 등의 사용자 정보를 저장한다고 가정해보자.  
레코드 하나만 저장하는 객체는 다음과 같은 형태일 것이다.  

```javascript
const user = {
    username: 'jsh',
    firstname: 'Sehun',
    lastName: 'Jeong',
    password: '0831'
}

// 객체 저장소 생성  
var store = db.createObjectStore("users", {keyPath: "username"});
```

#### 트랜잭션  
객체 저장소를 생성하는 단계 이후의 조작은 모두 '트랜잭션'을 통해 이루어진다.  
데이터를 읽거나 변경할 때마다 모든 변경사항을 트랜잭션으로 묶어서 처리한다.  
트랜잭션은 데이터베이스 객체의 transaction() 메서드로 생성한다.  

```javascript
const transaction = db.transaction();
```


#### 제한점  
IndexedDB의 한계점은 대부분 웹 스토리지의 한계와 정확히 일치한다.  
첫 번째, IndexedDB 데이터베이스는 페이지 소스 (프로토콜, 도메인, 포트)에 묶이므로 도메인 사이에 정보를 공유할 수 없다. 
두 번째, 소스별로 저장 가능한 데이터에 제한이 있다.  
파이어폭스 4 이상은 50MB, 크롬은 5MB로 제한한다.  


## 23.4 요약  

오프라인 웹 애플리케이션과 클라이언트 데이터 스토리지는 웹의 미래에서 중요한 부분을 차지할 것이다.
웹 스토리지 사용 시 자바스크립트에서도 상당한 데이터를 클라이언트에 저장 가능하다.  
데이터 캐시는 암호화되지 않으므로 민감한 정보를 저장해서는 안된다.  